In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,065 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,32

In [4]:
from operator import add
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

In [11]:
# Download the files
# !wget -q https://www.cs.utexas.edu/~kiat/datasets/flights.csv.bz2

!wget -q https://www.cs.utexas.edu/~kiat/datasets/flights-10k.csv

!wget -q https://www.cs.utexas.edu/~kiat/datasets/airports.csv

In [12]:
lines = sc.textFile("flights-10k.csv")
airports_lines = sc.textFile('airports.csv')

In [13]:
# Removing the Header Line from CSV file
linesHeader = lines.first()
#header = sc.parallelize([linesHeader])
#linesWithOutHeader = lines.subtract(header)

flights_raw = lines\
    .zipWithIndex()\
    .filter(lambda x: x[1] > 1)\
    .keys()

#flishgts = flights_lines\
#    .map(lambda x: x.split(','))
#flights = flights_raw.map(lambda p: (p[0], p[1] , p[2] , p[3], p[4] , p[5] , p[6], p[7] , p[8] , p[9], p[10], '0' if p[11] == '' else p[11], p[24] ))


flights = flights_raw\
    .map(lambda x: x.split(','))\
    .map(lambda x: (x[0], x[1], x[2], x[3], x[4],
                    x[5], x[6], x[7], x[8], x[9],
                    x[10], 0 if x[11]=='' else float(x[11]), x[12], x[13], x[14],
                    x[15], x[16], x[17], x[18], x[19],
                    x[20], x[21], x[22], x[23], x[24],
                    x[25], x[26], x[27], x[28], x[29], x[30]
))


airports_raw = airports_lines\
    .zipWithIndex()\
    .filter(lambda x:x[1]>1)\
    .keys()

airports = airports_raw\
    .map(lambda x: x.split(','))\
    .map(lambda x: (x[0], x[1], x[2], x[3]))\
    .filter(lambda x: x[0])


mainFlightsData = flights.map(lambda p: (p[0], p[1], p[2], p[3], p[4], p[5],
                                         p[6], p[7], p[8], p[9], p[10], p[11], p[24]))

mainFlightsData.take(1)

[('2015',
  '1',
  '1',
  '4',
  'AA',
  '2336',
  'N3KUAA',
  'LAX',
  'PBI',
  '0010',
  '0002',
  -8.0,
  '0')]

In [14]:
# number 6 is ORIGIN_AIRPORT
flightsFromBoston = mainFlightsData\
    .filter(lambda p: True if p[7] == "BOS" else False )

# Get the total number of Flights from BOS
# 107847 flights from Logan Airport in Boston
print(flightsFromBoston.count())

# Find the subset of flights departing on the weekend.
weekEndFlights = flightsFromBoston\
    .filter(lambda p: True if (int(p[3])==6 or int(p[3]) ==7) else False)
print(weekEndFlights.count()) # 26092

179
0


In [15]:
#Q1 Find a list of Origin Airports
result = flights\
    .map(lambda x:x[7])\
    .distinct()


# Save the RDD into a Folder
# This can be a path on Cloud Storage

# Just show one of the results
result.collect()

['DEN',
 'PDX',
 'MSP',
 'PHX',
 'PSE',
 'BQN',
 'ORD',
 'GEG',
 'ONT',
 'MCO',
 'BOS',
 'HIB',
 'MAF',
 'DFW',
 'IAH',
 'BRO',
 'BJI',
 'SGF',
 'IAD',
 'BUF',
 'JFK',
 'SMF',
 'AUS',
 'SBA',
 'SMX',
 'CLE',
 'GSP',
 'RDM',
 'CID',
 'TPA',
 'TYR',
 'RSW',
 'OAK',
 'PVD',
 'RNO',
 'PIT',
 'MIA',
 'TUL',
 'MSY',
 'OKC',
 'ATW',
 'MEM',
 'TYS',
 'CLT',
 'GRB',
 'ABE',
 'JAN',
 'FAR',
 'LEX',
 'JLN',
 'RDU',
 'MHK',
 'TLH',
 'BHM',
 'CAE',
 'TXK',
 'DTW',
 'EAU',
 'DLH',
 'FSD',
 'INL',
 'PAH',
 'ISN',
 'MFE',
 'BTV',
 'SPS',
 'SJT',
 'GCC',
 'OGG',
 'SJC',
 'GUC',
 'ORF',
 'MLU',
 'KOA',
 'LAR',
 'ITO',
 'GRI',
 'MDW',
 'MDT',
 'LIH',
 'TWF',
 'STC',
 'HPN',
 'MLB',
 'CIU',
 'DSM',
 'BLI',
 'ROW',
 'FWA',
 'ALB',
 'LNK',
 'CMI',
 'GTF',
 'GRR',
 'SUX',
 'DVL',
 'BUR',
 'DAB',
 'DAY',
 'GRK',
 'GJT',
 'BMI',
 'TVC',
 'LCH',
 'DBQ',
 'GCK',
 'GNV',
 'BRD',
 'DIK',
 'LBB',
 'BTM',
 'ELM',
 'PIH',
 'VEL',
 'HDN',
 'JMS',
 'ROA',
 'CHA',
 'EYW',
 'MYR',
 'CRW',
 'CHO',
 'GFK',
 'TOL',
 'OTZ',


In [ ]:
#Q2 Find a list of (Origin, Destination) pairs
result = flights\
    .map(lambda x: (x[7],x[8]))\
    .distinct()

# Save the RDD into a Folder
# This can be a path on Cloud Storage
# result.saveAsTextFile("Origin-Destination")

# Just show one of the results
result.take(1)

[('SEA', 'ANC')]

In [16]:
flights.filter(lambda x: x[1]=="1").map(lambda x: (x[7], x[11]) ).top(1, key=lambda x: x[1])

[('BOS', 1190.0)]

In [17]:
#Q3 Find the Origin airport which had the largest departure delay in the month of January
result = flights\
    .filter(lambda x: x[1]=="1")\
    .map(lambda x: (x[7], x[11]) )\
    .top(1, key=lambda x: x[1])

print(result)

[('BOS', 1190.0)]


In [18]:
#Q4 Find out which Airline has the largest delay on Weekends.
result = flights\
    .filter(lambda x: x[3]=="6" or x[3]=="7")\
    .map(lambda x: (x[4],x[11]) )\
    .top(1,lambda x: x[1])

print(result)

[]


In [19]:
#Q5 Which airport has the most cancellation of flights?
result = flights\
    .filter(lambda x: x[24]=="1")\
    .map(lambda x: (x[7],1))\
    .reduceByKey(add)\
    .top(1, lambda x: x[1])

print(result)

[('DFW', 153)]


In [20]:
flights.map(lambda x: (x[4], ((int(x[24])), 1))).reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).map(lambda x: (x[0], x[1][0]/x[1][1] * 100)).take(4)

[('DL', 0.0),
 ('NK', 2.1621621621621623),
 ('UA', 0.8849557522123894),
 ('B6', 0.0)]

In [21]:
#Q6 Find the percent of flights cancelled for each Airline.
result = flights\
    .map(lambda x: (x[4], ((int(x[24])), 1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1] * 100))

# Just show one of the results
result.take(1)


[('DL', 0.0)]

In [22]:
#Q7 Find the largest departure delay for each carrier
result = flights\
    .map(lambda x: (x[4], x[11]) )\
    .reduceByKey(max)

# Just show one of the results
result.take(1)

[('DL', 511.0)]

In [23]:
#Q8 Find the largest departure delay for each carrier for each month
result = flights\
    .map(lambda x: ((x[4], int(x[1])), x[11]) )\
    .reduceByKey(max)

# Just show one of the results
result.take(1)


[(('AA', 1), 1190.0)]

In [24]:
#Q9 For each carrier find the average Departure delay
result = flights \
    .map(lambda x: (x[4], (x[11], 1))) \
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1])) \

# Just show one of the results
result.take(1)


[('DL', 3.169750603378922)]

In [25]:
#Q10 For each carrier find the average Departure delay for each month
result = flights.map(lambda x: ((x[4], int(x[1])),(x[11],1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1]))

# Just show one of the results
result.take(1)


[(('AA', 1), 12.56616257088847)]

In [26]:
#Q11 Which date of year has the highest rate  of flight cancellations?
# Rate of flight cancellation is calculated by deviding number of canceled flights by total number of flights.
result = flights\
    .map(lambda x: ((x[0],x[1],x[2]), ((int(x[24])), 1)) )\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1]))\
    .top(1, lambda x:x[1])

print(result)


[(('2015', '1', '1'), 0.0392039203920392)]


In [27]:
flights.count()

9999

In [28]:
airports.count()

321

In [29]:
#Q12 Calculate the number of flights to each destination state
# For each carrier, for which state do they have the largest average delay?
# You will need the airline and airport data sets for this question.

result = flights\
    .map(lambda x: ( (x[4],x[8]), (x[11],1) ))\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
    .map(lambda x: (x[0][1], (x[0][0], x[1][0]/x[1][1]) ))\
    .leftOuterJoin(airports.map(lambda x: (x[0], x[3])))\
    .filter(lambda x: x[1][1] is not None)\
    .map(lambda x: (x[1][0][0], (x[1][1], x[1][0][1])))\
    .reduceByKey(lambda x,y: (x[0],x[1]) if x[1] > y[1] else (y[0],y[1]) )\
    .map(lambda x: (x[0], x[1][0]))

# Just show one of the results
result.collect()


[('UA', 'TX'),
 ('OO', 'MO'),
 ('US', 'CA'),
 ('WN', 'IA'),
 ('HA', 'CA'),
 ('DL', 'SD'),
 ('NK', 'NJ'),
 ('F9', 'MI'),
 ('B6', 'WA'),
 ('EV', 'TX'),
 ('MQ', 'VA'),
 ('AS', 'MO'),
 ('AA', 'HI'),
 ('VX', 'TX')]